In [1]:
# Импорт необходимых библиотек
import pickle
import numpy as np
import pandas as pd
import datetime
import re

from scipy.sparse import hstack
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier
import optuna

import torch
from transformers import DistilBertTokenizer, DistilBertModel

# Вступление

Данный ноутбук родился с целью доказать, что в соревновании Alice не этично было использовать другие модели, кроме LogReg. Так как это нарушает баланс справедливости, и не дает возможности новичкам побороться за призовые места. Основной идеей соревнования была борьба в способности генерировать признаки, а не во владении определенными моделями или инструментами. Цель работы не выжать максимум скора, а показать существенную разницу между различными вариантами. Чтож попробуем ее показать.

Рассмотрим следующие варианты:
1. TF-IDF + LogReg - классический вариант, который был рассмотрен в бейзлайне
2. TF-IDF + Catboost - линейная модель заменяется на бустинг
3. BERT + LogReg - признаки сайтов будем извлекать с помощью токенайзера берта, времянные признаки остаются без изменений.
4. BERT + Catboost - линейная модель заменяется на бустинг

Все предсказания будут загружены в песочницу и результаты будут сведены в одну таблицу.

Не зависимо от результа, постараюсь оставлять как можно больше комментариев на русском. Возможно, новичкам пригодится, хотя я и сам только учусь.

Поехали.

#  Загрузка данных и генерация признаков

Тут все стандартно

In [2]:
times = ['time'+str(i) for i in range(1, 11)]
sites = ['site'+str(i) for i in range(1, 11)]

In [3]:
# Загрузка датасетов
train_df = pd.read_csv('../input/ods-lincourse-alice/train.csv',
                       index_col='session_id', parse_dates=times)

test_df = pd.read_csv('../input/ods-lincourse-alice/test.csv',
                      index_col='session_id', parse_dates=times)

# Сортировка по времени
train_df = train_df.sort_values(by='time1')

# Заполняем пропуски нулями
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

Проверка, что загрузили. Советую проверять себя после каждого преобразования таблиц/таблицы, возможно с опытом в этом не будет необходимости, но так как я сам новичок, поэтому проверяю свои действия постоянно.

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253561 entries, 27554 to 11690
Data columns (total 21 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   site1   253561 non-null  int64         
 1   time1   253561 non-null  datetime64[ns]
 2   site2   253561 non-null  int64         
 3   time2   250098 non-null  datetime64[ns]
 4   site3   253561 non-null  int64         
 5   time3   246919 non-null  datetime64[ns]
 6   site4   253561 non-null  int64         
 7   time4   244321 non-null  datetime64[ns]
 8   site5   253561 non-null  int64         
 9   time5   241829 non-null  datetime64[ns]
 10  site6   253561 non-null  int64         
 11  time6   239495 non-null  datetime64[ns]
 12  site7   253561 non-null  int64         
 13  time7   237297 non-null  datetime64[ns]
 14  site8   253561 non-null  int64         
 15  time8   235224 non-null  datetime64[ns]
 16  site9   253561 non-null  int64         
 17  time9   233084 non-null  d

In [5]:
# Загрузим словарь веб-сайтов
with open(r"../input/ods-lincourse-alice/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)
site_dict['Unknown'] = 0

## Выделение признаков из времянных меток

Целью работы не является максимальный скор, поэтому пачка извлеченных признаков ниже предствалена для примера. Вы можете использовать свои признаки.

Выделим числовые признаки:
* длина посещения одного сайта (секунды)
* общее колиство проведенного времени за сесиию (секунды)
* и другие


А так же категориальные признаки, которые еще трансформируем get_dummies энкодером:
* количество посещенных сайтов
* номер часа
* выходной или будний день
* и другие

In [6]:
def add_num_ft (df):
  '''
  Функция выделяет числовые и категориальные признаки из времянных столбцов
  датасета 
  '''
  table = df.copy()

  # ЧИСЛОВЫЕ ПРИЗНАКИ
  # добавление времени посещения каждого сайта
  for i in range(1,10):
    table['diff_time'+str(i+1)+'-'+str(i)] = (table['time'+str(i+1)] - table['time'+str(i)]).apply(lambda x: x.seconds)
  
  # добавление общего времени сессии
  diff_list = list(set(table.columns.to_list()) - set(times) - set(sites) - set(['target']))
  table[diff_list] = table[diff_list].fillna(0).astype('int')
  table['total_time'] = table[diff_list].sum(axis=1)

  # отношение общего времени за сессию к медианному значению
  table['ratio_total_time'] = table['total_time'] / table['total_time'].median()

  # год и месяц в числовой форме
  table['yyyymm'] = table['time1'].apply(lambda x: 100 * x.year + x.month).astype(np.int64)

  # КАТЕГОРИАЛЬНЫЕ ПРИЗНАКИ
  # общее количество посещенных сайтов
  table['count_sites'] = table[sites].apply(lambda x: len(x.unique()), axis=1)

  # номер часа
  table['number_hour'] = table['time1'].apply(lambda x: x.hour)

  # выходной или будний день
  table['is_not_weekend'] = table['time1'].apply(lambda x: 1 if x.date().weekday() not in (5, 6) else 0)

  # это понедельник
  table['is_mon'] = table['time1'].apply(lambda x: x.dayofweek in [0]).astype(np.int64)

  # это вторник
  table['is_tue'] = table['time1'].apply(lambda x: x.dayofweek in [1]).astype(np.int64)

  # это пятница
  table['is_fri'] = table['time1'].apply(lambda x: x.dayofweek in [4]).astype(np.int64)

  # онлайн дни Алис
  table['online_day'] = table['time1'].apply(lambda x: x.dayofweek in [0,1,4,3,2]).astype(np.int64)

  # У Алис короче общее время сессии
  table['short'] = table['total_time'].map(lambda x: x < 39).astype(np.int64)

  # У Алис короче время первого сайта
  table['long'] = table['diff_time2-1'].map(lambda x: x > 8).astype(np.int64)

  # Онлайн часы Алис

  table['online_hour'] = table['time1'].apply(lambda x: x.hour in [9, 12, 13, 15, 16, 17, 18]).astype(np.int64)
  
  # # время дня - утро 6.00 - 12.00, день 12.00 - 18.00, 
  # #             вечер 18.00 - 22.00, ночь 22.00 - 6.00
  
  def period_of_day(time):
    # возвращает период суток
    t0 = time.round('T').time()
    
    t1 = datetime.time(6, 00)
    t2 = datetime.time(12, 00)
    t3 = datetime.time(18, 00)
    t4 = datetime.time(22, 00)
    
    if t2 > t0 >= t1 :
      return 1 # утро
    elif t3 > t0 >= t2:
      return 2 # день
    elif t4 > t0 >= t3:
      return 3 # вечер
    else:
      return 4 # ночь

  table['period_of_day'] = table['time1'].apply(period_of_day)

  # первый и последний сайт являются youtube
  youtube_ids = []

  for key in list(site_dict.keys()):
    if 'youtube' in key:
      youtube_ids.append(site_dict[key])
  def is_site(x, l):
    if x in l:
      return 1 
    return 0
  
  table['yb_start'] = table['site1'].apply(lambda x: is_site(x, youtube_ids))
  table['yb_end'] = table['site10'].apply(lambda x: is_site(x, youtube_ids))

  # убираем времянные колонки, они больше не нужны
  col_list = sorted(list(set(table.columns.to_list()) - set(times)))
  
  return table[col_list]

Применим нашу функцию к исходным датасетами

In [7]:
train_df_mod = add_num_ft(train_df)
test_df_mod = add_num_ft(test_df)
print(train_df_mod.shape, test_df_mod.shape)

(253561, 36) (82797, 35)


Отлично, получилось 25 колонок с новыми признаками ( -10шт на колонки сайтов и -1 на таргет). Теперь надо масштабировать значения для числовых колонок, и применить кодирование для категориальных

In [8]:
cat_col = ['count_sites', 'is_fri', 'is_mon', 'is_not_weekend', 'is_tue', 
           'long', 'number_hour', 'online_day', 'online_hour', 'period_of_day', 
           'short', 'yb_end', 'yb_start']
num_col = ['diff_time10-9', 'diff_time2-1', 'diff_time3-2', 'diff_time4-3', 
           'diff_time5-4', 'diff_time6-5', 'diff_time7-6', 'diff_time8-7', 
           'diff_time9-8', 'total_time', 'ratio_total_time', 'yyyymm']

In [9]:
train_df_mod = pd.get_dummies(data=train_df_mod, columns=cat_col, drop_first=True)
test_df_mod = pd.get_dummies(data=test_df_mod, columns=cat_col, drop_first=True)

In [10]:
scaler = StandardScaler()
scaler.fit(train_df_mod[num_col]) 
train_df_mod[num_col] = scaler.transform(train_df_mod[num_col])
test_df_mod[num_col] = scaler.transform(test_df_mod[num_col])

Новые признаки сделали из времянных колонок и трансформировали их. Эти признаки будут постоянными во всех вариантах и не будут меняться.

Дальше нужно извлечь признаки из колонок сайтов, вот тут будут применяться разные подходы и модели будут разные, поэтому ниже пойдет ветвление на варианты.

Так же дополнительно, чтобы сократить время расчетов, возьмем выборку из тренировочного датасета, и попробуем сбалансировать ее.

In [11]:
df_sample = train_df_mod.sample(n=3000, weights=1./train_df_mod.groupby('target')['target'].transform('count'), random_state=101).reset_index(drop=True)

In [12]:
df_sample['target'].value_counts(normalize=True)

0    0.6
1    0.4
Name: target, dtype: float64

Отлично в нашей выборке 40% Алис и 60% не Алис.

# Обучение моделей

## Вариант 1
TF-IDF + LogReg

Нужно склеить наши посещенные сайты, чтобы их передать в TF-IDF

In [13]:
df_sample[sites].fillna(0).to_csv('train_sessions_text.txt', 
                                 sep=' ', index=None, header=None)
test_df_mod[sites].fillna(0).to_csv('test_sessions_text.txt', 
                                sep=' ', index=None, header=None)

Инициализируем **TfidfVectorizer**

In [14]:
count_tf_idf = TfidfVectorizer(ngram_range=(1,10), analyzer='char_wb', max_features=50000)

In [15]:
with open('train_sessions_text.txt') as inp_train_file:
    X_train = count_tf_idf.fit_transform(inp_train_file)
with open('test_sessions_text.txt') as inp_test_file:
    X_test = count_tf_idf.transform(inp_test_file)

print(X_train.shape, X_test.shape)

(3000, 15667) (82797, 15667)


Теперь в данные матрицы добавим наши признаки извлеченные из столбцов времени

In [16]:
def add_to_sparce_marix(table, X_sparse):
    '''
    Фукция добавления признаков к разряженой матрице
    '''
    col_list = sorted(list(set(table.columns.to_list()) - set(sites) - set(['target'])))
    temp_array = table[col_list].values

    return hstack([X_sparse, temp_array]).tocsr()

In [17]:
X_full = add_to_sparce_marix(df_sample, X_train)
X_test = add_to_sparce_marix(test_df_mod, X_test)

print(X_full.shape, X_test.shape)

(3000, 15717) (82797, 15717)


Разобьем выборку на две части: тренировочную (70%), валидационную (30%)

In [18]:
border = len(df_sample)//100*70

In [19]:
X_train = X_full[:border]
X_valid = X_full[border:]
y_full = df_sample['target'].values
y_train = y_full[:border]
y_valid = y_full[border:]

Проверка

In [20]:
print(X_train.shape, X_valid.shape, y_train.shape, y_valid.shape)

(2100, 15717) (900, 15717) (2100,) (900,)


Обучаем модель

In [21]:
tscv = TimeSeriesSplit(n_splits=10)

In [22]:
%%time
logreg = LogisticRegression(solver='lbfgs', max_iter=1000, random_state=101)

tuned_parameters = {"C": np.logspace(-2, 2, 10)}

best_logreg = GridSearchCV(logreg, param_grid=tuned_parameters, scoring='roc_auc',
                           cv=tscv, n_jobs=-1).fit(X_train, y_train)
y_pred = best_logreg.predict_proba(X_valid)[:, 1]
print('***'*40)
print()
print('Лучшие параметры:')
print(best_logreg.best_params_)
print()
score = roc_auc_score(y_valid, y_pred)
print(score)

************************************************************************************************************************

Лучшие параметры:
{'C': 12.915496650148826}

0.9721228714741006
CPU times: user 3.59 s, sys: 3.71 s, total: 7.3 s
Wall time: 14.8 s


In [23]:
# Функция для записи предсказаний
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = range(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [24]:
# Делаем предсказания для теста
y_test = best_logreg.predict_proba(X_test)[:, 1]

# Сохрание файла с предсказаниями
write_to_submission_file(y_test, 'my_sub_var_1.csv')

Создадим таблицу, куда будем заносить результаты

In [25]:
result_df = pd.DataFrame(columns=['ML_name', 'ROC_AUC valid', 'ROC_AUC_public'])

In [26]:
result_df.loc[0]=['TF-IDF + LogReg', round(score, 6), '0,XXX'] # 0,XXX - значение полученное на паблике
result_df.head()

,ML_name,ROC_AUC valid,ROC_AUC_public
0,TF-IDF + LogReg,0.972123,"0,XXX"


## Вариант 2
TF-IDF + Catboost

В данном варианте заменим модель на бустинг, поддержим отечественного производителя =) и будем рассматривать Catboost. Никаких изменений данных, только меняем модель. Для подбора параметров воспользуемся **optuna**, она лучше, чем GridSearch или RandomSearch. Советую ее изучить и пользоваться ей.

In [27]:
def objective(trial):

  param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 4, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
        "used_ram_limit": "16gb",
        "eval_metric": "AUC",
        "random_state": 101,
        "logging_level": "Silent"
    }
    
  best_cat = CatBoostClassifier(**param)

  best_cat.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=20)
  y_pred = best_cat.predict_proba(X_valid)[:, 1]
  score = roc_auc_score(y_valid, y_pred)

  return score

Запускать поиск параметров не буду, они замимают пару часов. Возьмем параметры ниже.

In [28]:
# %%time
# study = optuna.create_study(study_name=f'catboost-seed{101}')
# study.optimize(objective, n_trials=100, n_jobs=-1, timeout=900)

Лучший скор:

In [29]:
# study.best_value

Лучшие параметры:

In [30]:
# study.best_params

In [31]:
# В одной из итераций поиска параметров, подобрались параметры ниже
# одни из лучших, кто не хочет ждать поиска может воспользоваться ими
# {'boosting_type': 'Plain',
#  'bootstrap_type': 'Bayesian',
#  'colsample_bylevel': 0.07809096596254919,
#  'depth': 4,
#  'objective': 'CrossEntropy'}

Создадим заново параметры и обучим модель, чтобы сделать предсказания для теста. Использовать сразу лучшую модель из **optuna** для предсказаний (типа как в GridSearch), как-то можно, но заморочено и я не разобрался. Проще заново обучить модель с найдеными параметрами и сделать предсказания.

In [32]:
best_params = {'boosting_type': 'Plain',
 'bootstrap_type': 'Bayesian',
 'colsample_bylevel': 0.07809096596254919,
 'depth': 4,
 'objective': 'CrossEntropy'}

In [33]:
# Задаем параметры, добавляя неоходимые параметры
params = best_params
params['eval_metric'] = 'AUC'
params['random_state'] = 101
params['logging_level'] = 'Silent'

In [34]:
# Обучаем модель с лучшими параметрами
best_cat = CatBoostClassifier(**params).fit(X_train, y_train, eval_set=(X_valid, y_valid), 
                                            early_stopping_rounds=20)
y_pred = best_cat.predict_proba(X_valid)[:, 1]
score = roc_auc_score(y_valid, y_pred)
print(score) # проверка, что скоры совпали, если был запущен поиск

0.9729669770738755


In [35]:
# Делаем предсказания для теста
y_test = best_cat.predict_proba(X_test)[:, 1]

# Сохрание файла с предсказаниями
write_to_submission_file(y_test, 'my_sub_var_2.csv')

Заносим очередной результат

In [36]:
result_df.loc[1]=['TF-IDF + Catboost', round(score, 6), '0,XXX'] # 0,XXX - значение полученное на паблике
result_df.head()

,ML_name,ROC_AUC valid,ROC_AUC_public
0,TF-IDF + LogReg,0.972123,"0,XXX"
1,TF-IDF + Catboost,0.972967,"0,XXX"


## Вариант 3

В этом варианте попробуем другой подход к извлечению признаков из столбцов сайтов. Воспользуемся токенайзером BERT. Сначала сведем все названия сайтов в одну строку. Далее передадим получившийся Series в функцию для генерации эмбендингов. Посмотрим, что из этого получится....

In [37]:
def inverse_dict(sites_dict):
    '''
    Функция для смены местами ключа и значения
    '''
    code_sites_dict = {}
    sites = list(sites_dict.items())
    for site in sites:
        code_sites_dict[site[1]] = site[0]
    return code_sites_dict

In [38]:
# заменяем коды сайтов на названия
df_sample[sites] = df_sample[sites].apply(lambda x: x.map(inverse_dict(site_dict)))
test_df_mod[sites] = test_df_mod[sites].apply(lambda x: x.map(inverse_dict(site_dict)))

In [39]:
# меняем тип данных, чтобы проще было их "сложить"
df_sample[sites] = df_sample[sites].astype('str')
test_df_mod[sites] = test_df_mod[sites].astype('str')

In [40]:
# обрезаем www у сайтов
df_sample[sites] = df_sample[sites].applymap(lambda x: re.sub("^\S*?\.*?www\S*?\.", '', x))
test_df_mod[sites] = test_df_mod[sites].applymap(lambda x: re.sub("^\S*?\.*?www\S*?\.", '', x))

In [41]:
# тут складываем в один стобец "all"
df_sample['all'] = df_sample[sites].apply(lambda x: ' '.join(x), axis=1)
test_df_mod['all'] = test_df_mod[sites].apply(lambda x: ' '.join(x), axis=1)

Проверка

In [42]:
df_sample['all'].head()

0    media-1.melty.fr media.melty.fr wat.tv med.wat...
1    um.simpli.fi foglio.basilic.io platform.linked...
2    id.google.fr google.fr clients1.google.fr goog...
3    deliv.leboncoin.fr static.leboncoin.fr lebonco...
4    oo.rfihub.com networkadvertising.org themig.co...
Name: all, dtype: object

In [43]:
test_df_mod['all'].head()

session_id
1    facebook.com s-static.ak.facebook.com apis.goo...
2    annotathon.org annotathon.org annotathon.org a...
3    safebrowsing-cache.google.com safebrowsing-cac...
4    fr.yahoo.com yahoo.fr ocsp.verisign.com ocsp.t...
5    sci.sciences.univ-bpclermont.fr sci.sciences.u...
Name: all, dtype: object

Инициализируем токенайзер и модель

In [44]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [45]:
def extract_features(tokenizer, model, series, flag_quantity = 0):
    '''
    Функция для создания эмбендингов
    '''

    # Преобразуем строку наших сайтов в номера токенов, возьмем половину макс длины токена
    tokenized = series.apply(lambda x: tokenizer.encode(x, max_length = 256, truncation=True, add_special_tokens=True))
    max_len = max([len(x) for x in tokenized])
    # Дополним нулями вектора до максимальной длины и создадим маску
    padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
    attention_mask = np.where(padded != 0, 1, 0)

    batch_size = 200
    embeddings = []
    
    # создадим флаг, на тот случай если размер таблицы нацело не делиться на размер батча
    quantity = 0
    if flag_quantity == 1:
        quantity = padded.shape[0] // batch_size + 1
    else:
        quantity = padded.shape[0] // batch_size        

    for i in range(quantity):
          # Преобразуем данные в формат тензоров
          batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
          attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

          # Для ускорения используем no_grad() те без градиена
          with torch.no_grad():
              # Чтобы получить эмбеддинги для батча, передадим модели данные и маску
              batch_embeddings = model(batch, attention_mask=attention_mask_batch)
          # Из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов
          embeddings.append(batch_embeddings[0][:,0,:].numpy())

    return pd.DataFrame(np.concatenate(embeddings))

Запускаем нашу функцию, изменить придется оба датасета, и семпл, на котором будем обучаться модель, и тест, для того чтобы модель смогла сделать предсказания.

In [46]:
%%time
df_ready_train = extract_features(tokenizer, model, df_sample['all'])

CPU times: user 20min 9s, sys: 5min 5s, total: 25min 15s
Wall time: 12min 48s


Разделим наш Series, потому что оперативной памяти, выделяемой кэгглом, не хватает. И будем постепенно извлекать признаки, сразу же формирую конечную таблицу test

In [47]:
test_df_mod['all'].shape

(82797,)

In [48]:
%%time
# Уверен, что есть элегантный способ, но в голову пришел только этот вариант
df_ready_test = pd.DataFrame()
temp_series = pd.Series()
for i in range(0,9):
    temp_series = test_df_mod['all'][i*10000:(i+1)*10000]
    if i < 8:
        df_ready_test_temp = extract_features(tokenizer, model, temp_series)
        df_ready_test = pd.concat([df_ready_test, df_ready_test_temp], ignore_index=True)
    elif i == 8:
        df_ready_test_temp = extract_features(tokenizer, model, temp_series, 1)
        df_ready_test = pd.concat([df_ready_test, df_ready_test_temp], ignore_index=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


CPU times: user 9h 18min 50s, sys: 2h 16min 22s, total: 11h 35min 12s
Wall time: 5h 52min 46s


Проверка

In [49]:
df_ready_test.shape

(82797, 768)

Удаляем лишние столбцы

In [50]:
df_sample = df_sample.drop(sites, axis=1)
test_df_mod = test_df_mod.drop(sites, axis=1)

In [51]:
df_sample = df_sample.drop(['all'], axis=1)
test_df_mod = test_df_mod.drop(['all'], axis=1)

Проверка

In [52]:
print(df_sample.shape, test_df_mod.shape)

(3000, 51) (82797, 50)


Объединение новых таблиц с таблицами, содержащими времянные признаки

In [53]:
df_ready_train = df_ready_train.join(df_sample)
df_ready_test = df_ready_test.join(test_df_mod.reset_index(drop=True))
print(df_ready_train.shape, df_ready_test.shape)

(3000, 819) (82797, 818)


Теперь разобьем семпл выборку на две части: тренировочную (70%), валидационную (30%)

In [54]:
df_train, df_valid = train_test_split(df_ready_train, test_size=0.3, random_state=101)

Проверка

In [55]:
print(df_train.shape, df_valid.shape)

(2100, 819) (900, 819)


Данные для обучения

In [56]:
features_train = df_train.drop(['target'], axis=1).values
target_train = df_train['target'].values

Данные для валидации

In [57]:
features_valid = df_valid.drop(['target'], axis=1).values
target_valid = df_valid['target'].values

Посмотрим, как с новыми признаками проявит себя **LogisticRegression**

In [58]:
%%time
logreg = LogisticRegression(solver='lbfgs', max_iter=2000, random_state=101)

tuned_parameters = {"C": np.logspace(-2, 2, 10)}

best_logreg = GridSearchCV(logreg, param_grid=tuned_parameters, scoring='roc_auc',
                           cv=tscv, n_jobs=-1).fit(features_train, target_train)
y_pred = best_logreg.predict_proba(features_valid)[:, 1]
print('***'*40)
print()
print('Лучшие параметры:')
print(best_logreg.best_params_)
print()
score = roc_auc_score(target_valid, y_pred)
print(score)

************************************************************************************************************************

Лучшие параметры:
{'C': 1.6681005372000592}

0.9505124819216499
CPU times: user 3.39 s, sys: 1.22 s, total: 4.62 s
Wall time: 37 s


In [59]:
# Делаем предсказания для теста
y_test = best_logreg.predict_proba(df_ready_test.values)[:, 1]

# Сохрание файла с предсказаниями
write_to_submission_file(y_test, 'my_sub_var_3.csv')

Заносим очередной результат

In [60]:
result_df.loc[2]=['BERT + LogReg', round(score, 6), '0,XXX'] # 0,XXX - значение полученное на паблике
result_df.head()

,ML_name,ROC_AUC valid,ROC_AUC_public
0,TF-IDF + LogReg,0.972123,"0,XXX"
1,TF-IDF + Catboost,0.972967,"0,XXX"
2,BERT + LogReg,0.950512,"0,XXX"


## Вариант 4

Аналогично Варианту 2 данные остаются без изменений, меняем только модель

In [61]:
def objective(trial):

  param = {
        "objective": trial.suggest_categorical("objective", ["Logloss", "CrossEntropy"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 4, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
        "used_ram_limit": "16gb",
        "eval_metric": "AUC",
        "random_state": 101,
        "logging_level": "Silent"
    }
    
  best_cat = CatBoostClassifier(**param)

  best_cat.fit(features_train, target_train, eval_set=(features_valid, target_valid), early_stopping_rounds=20)
  y_pred = best_cat.predict_proba(features_valid)[:, 1]
  score = roc_auc_score(target_valid, y_pred)

  return score

In [62]:
# %%time
# study = optuna.create_study(study_name=f'catboost-seed{101}')
# study.optimize(objective, n_trials=100, n_jobs=-1, timeout=900)

Лучший скор и лучшие параметры

In [63]:
# study.best_value

In [64]:
# study.best_params

In [65]:
best_params = {'boosting_type': 'Plain',
 'bootstrap_type': 'Bayesian',
 'colsample_bylevel': 0.07809096596254919,
 'depth': 4,
 'objective': 'CrossEntropy'}

In [66]:
# Задаем параметры
params = best_params
params['eval_metric'] = 'AUC'
params['random_state'] = 101
params['logging_level'] = 'Silent'

In [67]:
# Обучаем модель с лучшими параметрами
best_cat = CatBoostClassifier(**params).fit(features_train, target_train, eval_set=(features_valid, target_valid), 
                                            early_stopping_rounds=20)
y_pred = best_cat.predict_proba(features_valid)[:, 1]
score = roc_auc_score(target_valid, y_pred)
print(score) # проверка, что скоры совпали, если был запущен поиск

0.9611918086734159


In [68]:
# Делаем предсказания для теста
y_test = best_cat.predict_proba(df_ready_test.values)[:, 1]

# Сохрание файла с предсказаниями
write_to_submission_file(y_test, 'my_sub_var_4.csv')

In [69]:
result_df.loc[3]=['BERT + Catboost', round(score, 6), '0,XXX'] # 0,XXX - значение полученное на паблике

Глядя на результаты использовать BERT для выделения признаков было не очень хорошей идеей, справился он хуже чем TF-IDF. Или скорее я упустил какой-то важный момент, и не до конца изменил параметры. Рассмотрел его из спортивного интереса.

# Сравнение вариантов

Подведем итоги

In [70]:
result_df.head()

,ML_name,ROC_AUC valid,ROC_AUC_public
0,TF-IDF + LogReg,0.972123,"0,XXX"
1,TF-IDF + Catboost,0.972967,"0,XXX"
2,BERT + LogReg,0.950512,"0,XXX"
3,BERT + Catboost,0.961192,"0,XXX"


Существенная разница или нет, решать Вам дорогие читатели! Картинку со скорами песочницы загружу в комментарии.

Как видно из полученной таблицы, можно улучшить метрику используя другие инструменты/модели, но основные тезисы соревнования - соревнование по генерации признаков, соревнование для новичков. Иван пытался донести, что можно получить хороший результат и с простой моделью, нужно только "напрячь мозги" и сгенерировать пару хороших признаков, точно определяющие Алис. И те новички кому это удалось реализовать на LogReg - молодцы! Мне, к сожалению, мозгов не хватило для извлечения хороших признаков. Остальное пусть остается на совести "нечестных неновичков". На этом прощаюсь, Всем спасибо за потраченное время. И успехов в учебе!!!

P.S. Буду благодарен за поставленные плюсы.